In [64]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from fake_useragent import UserAgent
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium_stealth import stealth
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import re # Expresiones regulares 
import time
import pandas as pd
import random as rd

In [65]:
url = 'https://musicstax.com/'
service = Service(executable_path='./chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")

In [66]:
# pip3 install seleniumbase

from seleniumbase import Driver



# initialize the driver in GUI mode with UC enabled

driver = Driver(uc=True, headless=False)



# set the target URL

url = "https://musicstax.com/"



# open URL with a 6-second reconnect time to bypass the initial JS challenge

driver.uc_open_with_reconnect(url, reconnect_time=6)


In [67]:
playlists = pd.read_csv('playlist.csv')
playlists.iloc[57][0]

C:\Users\Nacho MPra\AppData\Local\Temp\ipykernel_34384\1601726479.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  playlists.iloc[57][0]


'Top 100 Last 120 days CO'

In [31]:
playlists.iloc[44][0]

C:\Users\Nacho MPra\AppData\Local\Temp\ipykernel_34384\2601093216.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  playlists.iloc[44][0]


'Top 100 Last 120 days ID'

In [68]:
def search_playlist(i):
    action = ActionChains(driver)
    search_box = driver.find_element(By.CSS_SELECTOR, 'body > div.container > header > div.menu-right > div.menu-container.menu-search.menu-search-bar > form > div > div > input') #searchbox
    time.sleep(1.3)
    action.move_to_element(search_box).click().send_keys(playlists.iloc[i][1]).perform()
    time.sleep(0.5)
    search_click = driver.find_element(By.CSS_SELECTOR, 'body > div.container > header > div.menu-right > div.menu-container.menu-search.menu-search-bar > form > div > button')
    time.sleep(0.5)
    search_click.click()

In [69]:
def scroll(n):
    current_height = driver.execute_script("return window.pageYOffset")

    while current_height < n:
        time.sleep(rd.uniform(0.5,1.2))
        driver.execute_script("window.scrollBy(0, 100)")
        current_height = driver.execute_script("return window.pageYOffset")
    return

In [70]:
def extract_info():
    html_source = driver.page_source
    soup = bs(html_source, 'html.parser')
    
    song_id = (soup.find(class_="song-discover-similar").a['href']).split('/')[-1]
    song_dict.update({'id': song_id})

    info_tech = soup.find_all(class_="song-fact-container-stat")

    keys_tech = ['tempo', 'key', 'loudness', 'beat']
    values_tech = [item.get_text().replace('\n', '').replace('\t', '').strip() for item in info_tech]

    tech_dict = {keys_tech[i]: values_tech[i] for i in range(len(keys_tech))}
    song_dict.update(tech_dict)

    info_music = (soup.find_all(class_="song-bar-stat"))

    keys_music = ['popularity', 'energy', 'danceability', 'positiveness','speechiness', 'liveness', 'acousticness','instrumentalness']
    values_music = [item.get_text().replace('\n', '').replace('\t', '').strip() for item in info_music]

    music_dict = {keys_music[i]: values_music[i] for i in range(len(keys_music))}

    song_dict.update(music_dict)


In [71]:
def move_to_song():
    actions = ActionChains(driver)

    # Obtener las coordenadas del elemento
    element_location = element.location
    current_location = driver.execute_script("return {x: window.scrollX, y: window.scrollY};")

    # Calcular la cantidad de pasos
    steps = 10
    x_step = (element_location['x'] - current_location['x']) / steps
    y_step = (element_location['y'] - current_location['y']) / steps

    # Realizar el desplazamiento en varios pasos pequeños
    for i in range(steps):
        driver.execute_script(f"window.scrollBy({x_step}, {y_step});")
        time.sleep(rd.uniform(0.1,0.5))  # Pequeña pausa entre cada desplazamiento
    driver.execute_script("window.scrollBy(0, -200);")
# Finalmente, mover el cursor al elemento
#actions.move_to_element(element).perform()
#time.sleep(0.2)


# Realiza varios desplazamientos pequeños para simular el uso de la rueda del ratón
#for i in range(10):
#    driver.execute_script("window.scrollBy(0, 20);")
#    time.sleep(0.1)


In [72]:
def click_song():
    click_s= driver.find_element(By.CSS_SELECTOR, css_selector)
    time.sleep(0.5)
    click_s.click()
    time.sleep(2.1)
    url_actual = driver.current_url
    # Comprueba si es la página de error 404
    if url_actual == "https://musicstax.com/404":
        time.sleep(0.5)  # Descanso breve antes de continuar
    else: 
        scroll(290)
    #return driver.current_url

In [73]:
#obtener el último índice para automatizar la extracción
def top_index():
    html_source_listado = driver.page_source
    soup_list = bs(html_source_listado, 'html.parser')
    last_song = (soup_list.find_all(class_="album-track-number"))
    last_index = int(last_song[-1].get_text().replace('\n', '').replace('\t', '').strip())*2
    return last_index

In [74]:
def artist_song(n = str):
    html_source_chart = driver.page_source
    soup = bs(html_source_chart, 'html.parser')
    tracks = soup.find_all('a', class_='album-track-container')

    # Iterar sobre los resultados y extraer información
    for track in tracks:
        # Obtener el número de pista
        track_number = track.find('div', class_='album-track-number').text.strip()
        
        # Comparar con el número de pista objetivo
        if track_number == n:
            # Obtener el título y el artista
            title = track.find('div', class_='album-song-title').find('u').contents[0].strip()
            artist = track.find('span', class_='album-song-artists').text.strip()
        
            # Devuelve título y artista como una tupla
            return f"{title} {artist}"

    # Si no se encuentra el número de pista, devuelve None
    return None

In [33]:
songs_details = []
for i in range(0, 25, 1):
    search_playlist(i)
    last_index = top_index()
    for i in range(2, last_index+2,2):
    #for i in range(40, 48,2):
        song_dict={}
        css_selector = str('body > div.container > div.container-middle > div > div.album-tracks > a:nth-child(' + str(i) + ') > div.album-track-number')
        element = driver.find_element(By.CSS_SELECTOR, css_selector)
        move_to_song()
        time.sleep(rd.uniform(0.5,1.2))
        click_song()
        time.sleep(rd.uniform(0.5,1.2))
        try:
            extract_info()
            print(f"Info Canción nº "+str(int(i/2))+" extraída")
        except AttributeError as e:
            print(f"Info Canción nº "+str(int(i/2))+" no disponible")
            time.sleep(0.5)
            #driver.back()
        time.sleep(rd.uniform(0.5,1.2))
        songs_details.append(song_dict)
        driver.back()
    #pasar a escalares todos las keys
df_details = pd.DataFrame(songs_details)
df_details
    

C:\Users\Nacho MPra\AppData\Local\Temp\ipykernel_21528\3962621495.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  action.move_to_element(search_box).click().send_keys(playlists.iloc[i][1]).perform()


KeyboardInterrupt: 

In [13]:
df_details.to_csv('top120Days_playlists_0_24-csv')

In [12]:
songs_details1 = []
for i in range(25, 40, 1):
    search_playlist(i)
    last_index = top_index()
    for i in range(2, last_index+2,2):
    #for i in range(40, 48,2):
        song_dict={}
        css_selector = str('body > div.container > div.container-middle > div > div.album-tracks > a:nth-child(' + str(i) + ') > div.album-track-number')
        element = driver.find_element(By.CSS_SELECTOR, css_selector)
        move_to_song()
        time.sleep(rd.uniform(0.5,1.2))
        click_song()
        time.sleep(rd.uniform(0.5,1.2))
        try:
            extract_info()
            print(f"Info Canción nº "+str(int(i/2))+" extraída")
        except AttributeError as e:
            print(f"Info Canción nº "+str(int(i/2))+" no disponible")
            time.sleep(0.5)
            #driver.back()
        time.sleep(rd.uniform(0.5,1.2))
        songs_details1.append(song_dict)
        driver.back()
    #pasar a escalares todos las keys
df_details1 = pd.DataFrame(songs_details1)
df_details1

Info Canción nº 1 extraída
Info Canción nº 2 extraída
Info Canción nº 3 extraída
Info Canción nº 4 extraída
Info Canción nº 5 extraída
Info Canción nº 6 extraída
Info Canción nº 7 extraída
Info Canción nº 8 extraída
Info Canción nº 9 extraída
Info Canción nº 10 extraída
Info Canción nº 11 extraída
Info Canción nº 12 extraída
Info Canción nº 13 extraída
Info Canción nº 14 extraída
Info Canción nº 15 extraída
Info Canción nº 16 extraída
Info Canción nº 17 extraída
Info Canción nº 18 extraída
Info Canción nº 19 extraída
Info Canción nº 20 no disponible
Info Canción nº 21 extraída
Info Canción nº 22 extraída
Info Canción nº 23 no disponible
Info Canción nº 24 extraída
Info Canción nº 25 extraída
Info Canción nº 26 extraída
Info Canción nº 27 extraída
Info Canción nº 28 extraída
Info Canción nº 29 extraída
Info Canción nº 30 extraída
Info Canción nº 31 extraída
Info Canción nº 32 extraída
Info Canción nº 33 extraída
Info Canción nº 34 extraída
Info Canción nº 35 extraída
Info Canción nº 36 

,id,tempo,key,loudness,beat,popularity,energy,danceability,positiveness,speechiness,liveness,acousticness,instrumentalness
0,2Lumsra3kuU61wXkEKzKaK,173 BPM,B min,-8.7db,4/4,82%,62%,47%,78%,3%,19%,3%,0%
1,1cOboCuWYI2osTOfolMRS6,113 BPM,F# Maj,-6.97db,3/4,91%,65%,69%,47%,8%,8%,50%,0%
2,5RvmmRCoZY1Yupl3PXgZm4,98 BPM,E Maj,-6.15db,3/4,74%,46%,57%,8%,3%,13%,25%,0%
3,5G2f63n7IPVPPjfNIGih7Q,113 BPM,D# Maj,-4.09db,4/4,91%,91%,67%,72%,6%,30%,10%,0%
4,2X7jfQnEMQYYmvkhr6mzS9,118 BPM,D min,-3.78db,4/4,69%,97%,77%,97%,4%,31%,2%,9%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,5xVToDFdml4kAgn2vIfJow,61 BPM,G min,-7.59db,4/4,46%,48%,64%,92%,29%,13%,82%,0%
1307,3Fcfwhm8oRrBvBZ8KGhtea,138 BPM,F min,-7.12db,4/4,72%,62%,49%,42%,3%,11%,10%,0%
1308,3BcKfPyEyWhd6R640RtGgQ,145 BPM,E Maj,-2.15db,4/4,44%,85%,92%,91%,17%,3%,0%,13%
1309,7gYGLJafbrlW0OyRbcEiUg,170 BPM,F# Maj,-15db,4/4,38%,43%,60%,35%,3%,12%,79%,88%


In [13]:
df_details1.to_csv('top120Days_playlists_25_39-csv')

In [ ]:
try:
    songs_details2 = []
    for i in range(40, 50, 1):
        search_playlist(i)
        print('Extrayendo...', playlists.iloc[i][0])
        last_index = top_index()
        for j in range(2, last_index+2,2):
        #for i in range(40, 48,2):
            song_dict={}
            css_selector = str('body > div.container > div.container-middle > div > div.album-tracks > a:nth-child(' + str(j) + ') > div.album-track-number')
            element = driver.find_element(By.CSS_SELECTOR, css_selector)
            move_to_song()
            time.sleep(rd.uniform(0.5,1.2))
            click_song()
            time.sleep(rd.uniform(0.5,1.2))
            try:
                extract_info()
                print(f"Info Canción nº "+str(int(j/2))+" extraída")
            except AttributeError as e:
                print(f"Info Canción nº "+str(int(j/2))+" no disponible")
                time.sleep(0.5)
                #driver.back()
            time.sleep(rd.uniform(0.5,1.2))
            songs_details2.append(song_dict)
            driver.back()
            df_details2 = pd.DataFrame(songs_details2)
            df_details2.to_csv('top120Days_playlists_40_49-csv')
        #pasar a escalares todos las keys
except KeyboardInterrupt:
    print("Script detenido manualmente")
finally:
    df_details2 = pd.DataFrame(songs_details2)
    df_details2.to_csv('top120Days_playlists_40_49-csv')

C:\Users\Nacho MPra\AppData\Local\Temp\ipykernel_34384\3962621495.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  action.move_to_element(search_box).click().send_keys(playlists.iloc[i][1]).perform()


Extrayendo... Top 100 Last 120 days IS
Info Canción nº 1 extraída
Info Canción nº 2 extraída
Info Canción nº 3 extraída
Info Canción nº 4 extraída
Info Canción nº 5 extraída
Info Canción nº 6 extraída
Info Canción nº 7 extraída
Info Canción nº 8 extraída
Info Canción nº 9 extraída
Info Canción nº 10 extraída
Info Canción nº 11 extraída
Info Canción nº 12 extraída
Info Canción nº 13 extraída
Info Canción nº 14 extraída
Info Canción nº 15 extraída
Info Canción nº 16 extraída
Info Canción nº 17 extraída
Info Canción nº 18 extraída
Info Canción nº 19 extraída
Info Canción nº 20 extraída
Info Canción nº 21 extraída
Info Canción nº 22 extraída
Info Canción nº 23 no disponible
Info Canción nº 24 extraída
Info Canción nº 25 extraída
Info Canción nº 26 extraída
Info Canción nº 27 extraída
Info Canción nº 28 extraída
Info Canción nº 29 extraída
Info Canción nº 30 extraída
Info Canción nº 31 extraída
Info Canción nº 32 extraída
Info Canción nº 33 extraída
Info Canción nº 34 extraída
Info Canción 

IndexError: list index out of range

In [45]:
songs_details3 = []
try:
    for i in range(44, 50, 1):
        search_playlist(i)
        print('Extrayendo...', playlists.iloc[i][0])
        last_index = top_index()
        for j in range(2, last_index+2,2):
        #for i in range(40, 48,2):
            song_dict={}
            css_selector = str('body > div.container > div.container-middle > div > div.album-tracks > a:nth-child(' + str(j) + ') > div.album-track-number')
            element = driver.find_element(By.CSS_SELECTOR, css_selector)
            move_to_song()
            time.sleep(rd.uniform(0.5,1.2))
            click_song()
            time.sleep(rd.uniform(0.5,1.2))
            try:
                extract_info()
                print(f"Info Canción nº "+str(int(j/2))+" extraída")
            except AttributeError as e:
                print(f"Info Canción nº "+str(int(j/2))+" no disponible")
                time.sleep(0.5)
                #driver.back()
            time.sleep(rd.uniform(0.5,1.2))
            songs_details3.append(song_dict)
            driver.back()
        #pasar a escalares todos las keys
except KeyboardInterrupt:
    print("Script detenido manualmente")
finally:
    df_details2 = pd.DataFrame(songs_details3)
    df_details2.to_csv('top120Days_playlists_44_49-csv')

C:\Users\Nacho MPra\AppData\Local\Temp\ipykernel_34384\3962621495.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  action.move_to_element(search_box).click().send_keys(playlists.iloc[i][1]).perform()


Extrayendo... Top 100 Last 120 days ID
Info Canción nº 1 extraída
Info Canción nº 2 extraída
Info Canción nº 3 extraída
Info Canción nº 4 extraída
Info Canción nº 5 extraída
Info Canción nº 6 extraída
Info Canción nº 7 extraída
Info Canción nº 8 extraída
Info Canción nº 9 extraída
Info Canción nº 10 extraída
Info Canción nº 11 extraída
Info Canción nº 12 extraída
Info Canción nº 13 extraída
Info Canción nº 14 extraída
Info Canción nº 15 extraída
Info Canción nº 16 extraída
Info Canción nº 17 extraída
Info Canción nº 18 extraída
Info Canción nº 19 no disponible
Info Canción nº 20 extraída
Info Canción nº 21 no disponible
Info Canción nº 22 extraída
Info Canción nº 23 no disponible
Info Canción nº 24 extraída
Info Canción nº 25 no disponible
Info Canción nº 26 extraída
Info Canción nº 27 extraída
Info Canción nº 28 extraída
Info Canción nº 29 extraída
Info Canción nº 30 extraída
Info Canción nº 31 extraída
Info Canción nº 32 extraída
Info Canción nº 33 no disponible
Info Canción nº 34 ex

In [57]:
songs_details4 = []
try:
    for i in range(50, 71, 1):
        search_playlist(i)
        print('Extrayendo...', playlists.iloc[i][0])
        last_index = top_index()
        for j in range(2, last_index+2,2):
        #for i in range(40, 48,2):
            song_dict={}
            css_selector = str('body > div.container > div.container-middle > div > div.album-tracks > a:nth-child(' + str(j) + ') > div.album-track-number')
            element = driver.find_element(By.CSS_SELECTOR, css_selector)
            move_to_song()
            time.sleep(rd.uniform(0.5,1.2))
            click_song()
            time.sleep(rd.uniform(0.5,1.2))
            try:
                extract_info()
                print(f"Info Canción nº "+str(int(j/2))+" extraída")
            except AttributeError as e:
                print(f"Info Canción nº "+str(int(j/2))+" no disponible")
                time.sleep(0.5)
                #driver.back()
            time.sleep(rd.uniform(0.5,1.2))
            songs_details4.append(song_dict)
            driver.back()
        #pasar a escalares todos las keys
except KeyboardInterrupt:
    print("Script detenido manualmente")
finally:
    df_details4 = pd.DataFrame(songs_details4)
    df_details4.to_csv('top120Days_playlists_50_70-csv')

C:\Users\Nacho MPra\AppData\Local\Temp\ipykernel_34384\3962621495.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  action.move_to_element(search_box).click().send_keys(playlists.iloc[i][1]).perform()


Extrayendo... Top 100 Last 120 days FI
Info Canción nº 1 extraída
Info Canción nº 2 extraída
Info Canción nº 3 extraída
Info Canción nº 4 extraída
Info Canción nº 5 extraída
Info Canción nº 6 extraída
Info Canción nº 7 extraída
Info Canción nº 8 extraída
Info Canción nº 9 extraída
Info Canción nº 10 no disponible
Info Canción nº 11 extraída
Info Canción nº 12 extraída
Info Canción nº 13 extraída
Info Canción nº 14 extraída
Info Canción nº 15 extraída
Info Canción nº 16 extraída
Info Canción nº 17 extraída
Info Canción nº 18 extraída
Info Canción nº 19 extraída
Info Canción nº 20 extraída
Info Canción nº 21 extraída
Info Canción nº 22 extraída
Info Canción nº 23 extraída
Info Canción nº 24 extraída
Info Canción nº 25 extraída
Info Canción nº 26 extraída
Info Canción nº 27 extraída
Info Canción nº 28 extraída
Info Canción nº 29 extraída
Info Canción nº 30 no disponible
Info Canción nº 31 extraída
Info Canción nº 32 extraída
Info Canción nº 33 extraída
Info Canción nº 34 extraída
Info Can

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"body > div.container > div.container-middle > div > div.album-tracks > a:nth-child(100) > div.album-track-number"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7BCAA80D5+2992373]
	(No symbol) [0x00007FF7BC73BFD0]
	(No symbol) [0x00007FF7BC5D590A]
	(No symbol) [0x00007FF7BC62926E]
	(No symbol) [0x00007FF7BC62955C]
	(No symbol) [0x00007FF7BC6727D7]
	(No symbol) [0x00007FF7BC64F3AF]
	(No symbol) [0x00007FF7BC66F584]
	(No symbol) [0x00007FF7BC64F113]
	(No symbol) [0x00007FF7BC61A918]
	(No symbol) [0x00007FF7BC61BA81]
	GetHandleVerifier [0x00007FF7BCB06A2D+3379789]
	GetHandleVerifier [0x00007FF7BCB1C32D+3468109]
	GetHandleVerifier [0x00007FF7BCB10043+3418211]
	GetHandleVerifier [0x00007FF7BC89C78B+847

In [75]:
songs_details5 = []
try:
    for i in range(57, 71, 1):
        search_playlist(i)
        print('Extrayendo...', playlists.iloc[i][0])
        last_index = top_index()
        for j in range(2, last_index+2,2):
        #for i in range(40, 48,2):
            song_dict={}
            css_selector = str('body > div.container > div.container-middle > div > div.album-tracks > a:nth-child(' + str(j) + ') > div.album-track-number')
            element = driver.find_element(By.CSS_SELECTOR, css_selector)
            move_to_song()
            time.sleep(rd.uniform(0.5,1.2))
            click_song()
            time.sleep(rd.uniform(0.5,1.2))
            try:
                extract_info()
                print(f"Info Canción nº "+str(int(j/2))+" extraída")
            except AttributeError as e:
                print(f"Info Canción nº "+str(int(j/2))+" no disponible")
                time.sleep(0.5)
                #driver.back()
            time.sleep(rd.uniform(0.5,1.2))
            songs_details5.append(song_dict)
            driver.back()
        #pasar a escalares todos las keys
except KeyboardInterrupt:
    print("Script detenido manualmente")
finally:
    df_details5 = pd.DataFrame(songs_details5)
    df_details5.to_csv('top120Days_playlists_57_70-csv')

C:\Users\Nacho MPra\AppData\Local\Temp\ipykernel_34384\3962621495.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  action.move_to_element(search_box).click().send_keys(playlists.iloc[i][1]).perform()


Extrayendo... Top 100 Last 120 days CO
Info Canción nº 1 extraída
Info Canción nº 2 extraída
Info Canción nº 3 extraída
Info Canción nº 4 extraída
Info Canción nº 5 extraída
Info Canción nº 6 extraída
Info Canción nº 7 extraída
Info Canción nº 8 extraída
Info Canción nº 9 extraída
Info Canción nº 10 extraída
Info Canción nº 11 extraída
Info Canción nº 12 extraída
Info Canción nº 13 extraída
Info Canción nº 14 extraída
Info Canción nº 15 extraída
Info Canción nº 16 extraída
Info Canción nº 17 extraída
Info Canción nº 18 extraída
Info Canción nº 19 extraída
Info Canción nº 20 extraída
Info Canción nº 21 extraída
Info Canción nº 22 extraída
Info Canción nº 23 extraída
Info Canción nº 24 no disponible
Info Canción nº 25 extraída
Info Canción nº 26 extraída
Info Canción nº 27 extraída
Info Canción nº 28 extraída
Info Canción nº 29 no disponible
Info Canción nº 30 extraída
Info Canción nº 31 extraída
Info Canción nº 32 extraída
Info Canción nº 33 extraída
Info Canción nº 34 extraída
Info Can

In [ ]:
songs_details = []
for i in range(0, 72, 1):
    search_playlist(i)
    last_index = top_index()
    #for i in range(2, last_index+2,2):
    for i in range(42, 48,2):
        song_dict={}
        css_selector = str('body > div.container > div.container-middle > div > div.album-tracks > a:nth-child(' + str(i) + ') > div.album-track-number')
        element = driver.find_element(By.CSS_SELECTOR, css_selector)
        move_to_song()
        #obtener el último índice para automatizar la extracción
        html_source_chart = driver.page_source
        soup_chart = bs(html_source_chart, 'html.parser')
        artist_song(str(int(i/2)))
        time.sleep(rd.uniform(0.5,1.2))
        url_actual=click_song()
        if url_actual == "https://musicstax.com/404":
            print(f"Página 404 encontrada para la Canción nº {i + 1}. Saltando...")
            time.sleep(0.5)  # Descanso breve antes de continuar
        else:
            extract_info()
            print(f"Info Canción nº "+str(int(i/2))+" extraída")
        time.sleep(rd.uniform(0.5,1.2))
        songs_details.append(song_dict)
        driver.back()
    #pasar a escalares todos las keys
df_details = pd.DataFrame(songs_details)
df_details

    

Info Canción nº 21 extraída


In [14]:
url_actual = driver.current_url
url_actual

'https://musicstax.com/404'

In [12]:
html_source = driver.page_source
soup = bs(html_source, 'html.parser')
    
song_id = (soup.find(class_="song-discover-similar").a['href']).split('/')[-1]
song_id

AttributeError: 'NoneType' object has no attribute 'a'

In [134]:
search_text = artist_song("22")
click_s= driver.find_element(By.CSS_SELECTOR, 'body > div.container > div.container-middle > div > div.album-tracks > a:nth-child(44) > div.album-track-number')
time.sleep(0.5)
click_s.click()
action = ActionChains(driver)
search_song = driver.find_element(By.CSS_SELECTOR, 'body > div.container > div.container-middle > div > form > input')
time.sleep(2.3)
action.move_to_element(search_song).click().send_keys(search_text).perform()
time.sleep(1.5)
search_song.submit()
time.sleep(1.7)
click_song('div.song-details:nth-child(3) > a:nth-child(3)')



In [132]:
a = 'body > div.container > div.container-middle > div > div.container-search > div.search-tracks-container > div:nth-child(3) > a.song-details-right.search-details-right'
b = 'div.song-details:nth-child(3) > a:nth-child(3)'

El scrappeo de toda la página debe ser:
sobre un for para ir extrayendo por playlist:
playlists.iloc[i][1] i in range 1-72
    for cada playlist:
        scroll(x)
        for cancación (body > div.container > div.container-middle > div > div.album-tracks > a:nth-child(i) > div.album-song-title) i in range (saltos de 2 en 2, sólo pares)
        checkear len get.text último elemento de: class='album-track-number" (range(2, (último album track number*2)+2, saltos de 2)
        vaciar song_dict = {}
        extraer
        añadir song_dict as row in df (https://sparkbyexamples.com/pandas/how-to-append-row-to-pandas-dataframe/)
        driver.back()


/html/body/div[3]/div[2]/div[1]/div[4]/a[1]
/html/body/div[3]/div[2]/div[1]/div[4]/a[2]
.
.
.
/html/body/div[3]/div[2]/div[1]/div[4]/a[94]
HAY QUE ITERAR SOBRE EL NÚMERO FINAL, EN FUNCIÓN DEL LEN DE LA PLAYLIST

In [101]:
driver.back()

In [12]:
driver.quit()

Guion presentación:

Presentar top 3 tonalidades con alguna melodía, que nos evoca (o contraponer la más comun en mayor y menor)
Igual con los beats, mostar un beat a bpm más rápido y beat bpm más lento

ideas viz
-regresión linea tonalidades y beat
-hay algún país que potencia modos menores/positiveness
-popularidad vs energía
-duración y posición en el top (si no afecta, explicación como se utilizan fragmentos, puede no tener tanta importancia)
-recordar pd.value_counts(normalize=True o False) para tener valores absolutos o relativos de categorías
-definir escala de tempo:
    Largo: muy lento (20 bpm).
    Lento: lento (40 – 60 bpm).
    Grave: lento y solemne (40 – 43 bpm).
    Adagio: lento y majestuoso (66 – 76 bpm).
    Andante: al paso, tranquilo, un poco vivaz (76 – 108 bpm).
    Moderato: moderado, como su propio nombre indica (80 – 108 bpm).
    Allegretto: un poco animado(96 – 112 bpm).
    Allegro: animado y rápido (110 – 168 bpm).
    Vivace: rápido (121 – 140 bpm).
    Presto: muy rápido (168 – 200 bpm).
    Prestissimo: rapidísimo (más de 200 bpm).
-
